In [7]:
import pandas as pd
with open("../../data/raw/LgaRankings_27_Offences.xlsx", "rb") as f:
    df_dict = pd.read_excel(f, sheet_name=None, encoding='latin-1', skiprows=0, header=[4,5], na_values=['nc'], skip_footer=7)


In [8]:
uns = pd.DataFrame(df_dict['Assault - domestic violence'].unstack())

uns.index = uns.index.rename(["Year","Type", "LGA"])
uns.columns = ['Assault - domestic violence']
uns

Assault - domestic violence
Year Type  LGA                                                 
2013 Total Albury                                         276.0
           Armidale Regional                              199.0
           Ballina                                        141.0
           Balranald                                        9.0
           Bathurst Regional                              195.0
           Bayside                                        386.0
           Bega Valley                                    104.0
           Bellingen                                       25.0
           Berrigan                                        24.0
           Blacktown                                     1866.0
           Bland                                           26.0
           Blayney                                         21.0
           Blue Mountains                                 242.0
           Bogan                                           18.0
           Bourke                                         139.0
           Brewarrina                                     104.0
           Broken Hill                                    176.0
           Burwood                                         74.0
           Byron                                          114.0
           Cabonne                                         33.0
           Camden                                         158.0
           Campbelltown                                   966.0
           Canada Bay                                     135.0
           Canterbury-Bankstown                          1311.0
           Carrathool                                      15.0
           Central Coast                                 1179.0
           Central Darling                                137.0
           Cessnock                                       384.0
           Clarence Valley                                220.0
           Cobar                                           39.0
...                                                         ...
2017 Rank  Shoalhaven                                      74.0
           Singleton                                       45.0
           Snowy Monaro Regional                           87.0
           Snowy Valleys                                   47.0
           Strathfield                                     80.0
           Sutherland Shire                                97.0
           Sydney                                           NaN
           Tamworth Regional                               25.0
           Temora                                          53.0
           Tenterfield                                     31.0
           The Hills Shire                                112.0
           Tweed                                           81.0
           Unincorporated Far West                          NaN
           Upper Hunter Shire                              99.0
           Upper Lachlan Shire                            116.0
           Uralla                                          71.0
           Wagga Wagga                                     20.0
           Walcha                                          85.0
           Walgett                                          1.0
           Warren                                           NaN
           Warrumbungle Shire                              35.0
           Waverley                                        96.0
           Weddin                                          66.0
           Wentworth                                        7.0
           Willoughby                                     117.0
           Wingecarribee                                   98.0
           Wollondilly                                     76.0
           Wollongong                                      82.0
           Woollahra                                      105.0
           Yass Valley                                    106.0

[19

In [9]:
cleaned_list = []

for key in df_dict:
    unstacked = pd.DataFrame(df_dict[key].unstack())

    unstacked.index = unstacked.index.rename(["Year","Type", "LGA"])
    unstacked.columns = [key]
    cleaned_list.append(unstacked)
    
final_df = cleaned_list[0]
for i in range(1,len(cleaned_list)):
    final_df = pd.merge(final_df, cleaned_list[i], how='inner', left_index=True, right_index=True)


In [18]:
output_df = final_df.reset_index()
output_df = output_df[output_df["Type"] == 'Rate per 100,000 population']
output_df = output_df.dropna()
output_df.loc[output_df["LGA"] == "Greater Hume Shire", "LGA"] = "Greater Hume"
output_df = output_df.loc[output_df["LGA"] != "Lord Howe Island"]

In [99]:
nonzeroes = output_df.astype(bool).sum(axis=0).sort_values(ascending=False)
nonzeroes = nonzeroes[nonzeroes== 602]
nonzeroes = nonzeroes.drop(['Type','LGA', 'Year'])
nonzeroes

Fraud                              602
Harassment threatening             602
Assault - domestic violence        602
Assault - non-domestic violence    602
Sexual Offences                    602
Break and enter dwelling           602
Break and enter non-dwelling       602
Steal from dwelling                602
Malicious damage to property       602
Prohibited weapons offences        602
Other theft                        602
Trespass                           602
dtype: int64

In [24]:
output_df.to_csv("cleaned_target.csv")

In [105]:
import numpy as np
variance = output_df.var()
corr = output_df.loc[:,nonzeroes.index].corr()
while len(corr) > 5:
    maxvals = []
    argmaxs = []
    for i,x in corr.iterrows():
        argmax = 0
        maxval = 0

        for j, y in enumerate(x[1:]):
            if y > maxval and y != 1.0:
                maxval = y
                argmax = j
        #print(i, maxval, x.index[argmax+1])
        maxvals.append(maxval)
        argmaxs.append(argmax)

    #the most correlated - drop the one with less variance
    mostcorridx = np.argmax(maxvals)
    mostcorr = corr.index[mostcorridx]
    mostcorr_other = corr.index[argmaxs[mostcorridx]+1]
    if variance[mostcorr] > variance[mostcorr_other]:
        corr = corr.drop(mostcorr_other)
        corr = corr.drop(mostcorr_other, axis=1)
        print("Dropping "+mostcorr_other)
    else:
        corr = corr.drop(mostcorr)
        corr = corr.drop(mostcorr, axis=1)      
        print("Dropping "+mostcorr)

Dropping Malicious damage to property
Dropping Assault - domestic violence
Dropping Break and enter dwelling
Dropping Assault - non-domestic violence
Dropping Break and enter non-dwelling
Dropping Harassment threatening
Dropping Trespass


In [106]:

corr

,Fraud,Sexual Offences,Steal from dwelling,Prohibited weapons offences,Other theft
Fraud,1.000000,-0.099326,0.036429,-0.191136,0.507591
Sexual Offences,-0.099326,1.000000,0.576855,0.578929,0.356578
Steal from dwelling,0.036429,0.576855,1.000000,0.571703,0.608549
Prohibited weapons offences,-0.191136,0.578929,0.571703,1.000000,0.332354
Other theft,0.507591,0.356578,0.608549,0.332354,1.000000


In [64]:
corr = output_df.iloc[:,1:].corr()

In [65]:
corr

,Assault - domestic violence,Assault - non-domestic violence,Sexual Offences,Robbery,Break and enter dwelling,Break and enter non-dwelling,Motor vehicle theft,Steal from motor vehicle,Steal from retail store,Steal from dwelling,...,Possession use of cannabis,Prohibited weapons offences,Trespass,Offensive conduct,Offensive language,Liquor offences,Breach AVO,Breach bail conditions,Resist or hinder officer,Transport regulatory offences
Assault - domestic violence,1.000000,0.785583,0.613775,0.257866,0.759894,0.669397,0.609851,0.590046,0.351364,0.722288,...,0.299827,0.648546,0.681526,0.461449,0.683876,0.105621,0.883122,0.669175,0.638837,-0.121318
Assault - non-domestic violence,0.785583,1.000000,0.648567,0.446734,0.714959,0.614345,0.564927,0.664433,0.551324,0.766689,...,0.461772,0.538755,0.660154,0.718614,0.662776,0.290404,0.742882,0.641874,0.761278,-0.028978
Sexual Offences,0.613775,0.648567,1.000000,0.106254,0.507208,0.537754,0.377646,0.398427,0.215485,0.576855,...,0.284490,0.578929,0.541816,0.458408,0.520228,0.181461,0.626888,0.397740,0.538687,-0.222665
Robbery,0.257866,0.446734,0.106254,1.000000,0.374122,0.144963,0.452416,0.459515,0.606985,0.252297,...,0.255091,0.048714,0.077106,0.263560,0.140501,0.103717,0.184163,0.455195,0.310621,0.433420
Break and enter dwelling,0.759894,0.714959,0.507208,0.374122,1.000000,0.756116,0.738350,0.767215,0.468647,0.785481,...,0.253510,0.528461,0.646554,0.397159,0.577230,0.050855,0.760316,0.616928,0.534429,-0.154122
Break and enter non-dwelling,0.669397,0.614345,0.537754,0.144963,0.756116,1.000000,0.590746,0.586163,0.225036,0.697622,...,0.260947,0.577598,0.684353,0.406405,0.600541,0.144785,0.668924,0.440660,0.469412,-0.254662
Motor vehicle theft,0.609851,0.564927,0.377646,0.452416,0.738350,0.590746,1.000000,0.757122,0.449796,0.621679,...,0.223932,0.406524,0.486736,0.244438,0.348903,-0.040909,0.596519,0.539612,0.403962,0.020646
Steal from motor vehicle,0.590046,0.664433,0.398427,0.459515,0.767215,0.586163,0.757122,1.000000,0.570121,0.692576,...,0.240908,0.348608,0.524945,0.382728,0.417549,0.096363,0.581282,0.564831,0.457660,0.056066
Steal from retail store,0.351364,0.551324,0.215485,0.606985,0.468647,0.225036,0.449796,0.570121,1.000000,0.406647,...,0.344051,0.161117,0.218969,0.362445,0.248909,0.133854,0.335824,0.624552,0.447769,0.297996
Steal from dwelling,0.722288,0.766689,0.576855,0.252297,0.785481,0.697622,0.621679,0.692576,0.406647,1.000000,...,0.341587,0.571703,0.670955,0.551886,0.580932,0.200012,0.737665,0.499905,0.583631,-0.226692
